# Demo

## Load Environment Vars

In [1]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

## Data Models

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Subcategory(BaseModel):
    """A project subcategory."""
    
    id: int = Field(description="The subcategory id.")
    name: str = Field(description="The subcategory name.")

class ReprovationReason(BaseModel):
    """A project reprovation reason."""
    
    id: int = Field(description="The reprovation reason id.")
    name: str = Field(description="The reprovation reason name.")
    description: str = Field(description="The reprovation reason description.")
    other: bool = Field(description="Whether the reprovation reason is an 'other' reason.", default=False)

class ProjectInput(BaseModel):
    """The project data defined by the client (may need changes)."""
    
    subcategory: int = Field(description="The subcategory of the project.")
    title: str = Field(description="The title of the project.")
    description: str = Field(description="The description of the project.")
    abilities: list[str] | None = Field(description="The abilities desired for the project.")

    def format(self):
        abilities_section = ('\n\nHabilidades: ' + ', '.join(self.abilities)) if len(self.abilities or []) else ''
        return f'{self.title}{abilities_section}\n\n{self.description}'
    
class ProjectPrediction(BaseModel):
    """The predicted project data based on the input. Some fields might be left undefined depending on the project being approved or not."""
    
    approve: bool = Field(description="Whether the project should be approved or not.")
    subcategory: int | None = Field(description="The subcategory of the project (when approved).")
    title: str | None = Field(description="The title of the project (when approved).", min_length=25, max_length=75)
    description: str | None = Field(description="The description of the project (when approved).", min_length=50, max_length=2000)
    reprovation_reason: int | None = Field(description="The reason for the project to be reproved (when reproved).")
    reprovation_comment: str | None = Field(description="The comment for the reprovation (when reproved).")
    
class ProjectInfo(BaseModel):
    """Additional information about the project."""
    
    reasoning: str = Field(description="The reasoning behind the prediction.")
    confidence: float = Field(description="The confidence of the prediction.", ge=0, le=1)
    bot_generated: float = Field(description="How likely is for the project to be bot generated.", ge=0, le=1)
    
    
class ProjectOutput(BaseModel):
    """The predicted project data based on the input, along with additional information."""
    
    prediction: ProjectPrediction = Field(description="The predicted project data based on the input.")
    info: ProjectInfo = Field(description="Additional information about the project.")
    
    
class ProjectApprovation(BaseModel):
    """The project input and prediction for the approvation process."""
    
    input: ProjectInput = Field(description="The project input.")
    prediction: ProjectPrediction = Field(description="The predicted project data based on the input.")

## Create Store

In [3]:
from langchain_core.vectorstores import VectorStore
import typing

T = typing.TypeVar('T')

class MyVectorStores:
    def __init__(
            self,
            subcategories: VectorStore,
            reprovation_reasons: VectorStore,
            project_approvation: VectorStore):
        self.subcategories = subcategories
        self.reprovation_reasons = reprovation_reasons
        self.project_approvation = project_approvation

class VectorStoreHandler(typing.Generic[T]):
    def __init__(self, store: VectorStore, json_parser: typing.Callable[[str], T]):
        self.store = store
        self.json_parser = json_parser
        
    def get_by_id(self, id: int):
        data = self.store.get(ids=[str(id)])
        metadatas = data['metadatas']
        metadata = metadatas[0] if len(metadatas or []) else None
        json_data = metadata['json'] if metadata else None
        data = self.json_parser(json_data) if json_data else None
        return data
    
    def similarity_search(
            self, 
            query: str, 
            k: int = 5):
        documents = self.store.similarity_search(query, k=k)
        return list(map(lambda d: self.json_parser(d.metadata['json']), documents))
    
    async def asimilarity_search(
            self, 
            query: str, 
            k: int = 5):
        documents = await self.store.asimilarity_search(query, k=k)
        return list(map(lambda d: self.json_parser(d.metadata['json']), documents))
    
    def max_marginal_relevance_search(
            self, 
            query: str, 
            k: int = 5):
        documents = self.store.max_marginal_relevance_search(query, k=k)
        return list(map(lambda d: self.json_parser(d.metadata['json']), documents))
    
    async def amax_marginal_relevance_search(
            self, 
            query: str, 
            k: int = 5):
        documents = await self.store.amax_marginal_relevance_search(query, k=k)
        return list(map(lambda d: self.json_parser(d.metadata['json']), documents))

class MyVectorStoreHandler:
    def __init__(self, stores: MyVectorStores):
        self.subcategories = VectorStoreHandler(store=stores.subcategories, json_parser=Subcategory.parse_raw)
        self.reprovation_reasons = VectorStoreHandler(store=stores.reprovation_reasons, json_parser=ReprovationReason.parse_raw)
        self.project_approvation = VectorStoreHandler(store=stores.project_approvation, json_parser=ProjectApprovation.parse_raw)

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

data_dir = "./data/chroma_db"
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

def create_store(collection_name):
    return Chroma(persist_directory=data_dir, collection_name=collection_name, embedding_function=embedding_function)

def create_stores():
    stores = MyVectorStores(
        subcategories=create_store("subcategories"),
        reprovation_reasons=create_store("reprovation_reasons"),
        project_approvation=create_store("project_approvation"),
    )
    stores_handler = MyVectorStoreHandler(stores)
    return stores, stores_handler

stores, stores_handler = create_stores()

c:\Users\Asus\anaconda3\envs\dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Asus\anaconda3\envs\dev\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Examples

In [5]:
def get_subcategories():
    return [
        Subcategory(id=1, name="Design gráfico"),
        Subcategory(id=2, name="Desenvolvimento Web"),
        Subcategory(id=3, name="Redação"),
        Subcategory(id=4, name="Edição de Vídeo"),
        Subcategory(id=5, name="Tradução"),
        Subcategory(id=6, name="Assistente Virtual"),
        Subcategory(id=7, name="Animação"),
        Subcategory(id=8, name="Marketing Digital"),
        Subcategory(id=9, name="Ilustração"),
    ]

def get_reprovation_reasons():
    return [
        ReprovationReason(id=1, name="Outro Motivo", description="", other=True),
        ReprovationReason(id=2, name="Projeto Duplicado", description="Você publicou um projeto muito parecido ou idêntico a esse a pouco tempo"),
        ReprovationReason(id=3, name="Proposta para Trabalhar", description="Se o seu objetivo é encontrar trabalhos, por favor, altere o seu perfil para o de Freelancer"),
        ReprovationReason(id=4, name="Conteúdo Não Permitido", description="Há conteúdo não permitido na descrição do projeto (informações de contato, orçamento, solicitação de testes não remunerados, etc)"),
        ReprovationReason(id=5, name="Projeto Confuso", description="Não está claro o que é pedido no projeto"),
        ReprovationReason(id=6, name="Poucos Detalhes", description="Os freelancers precisam saber detalhes para que possam enviar as suas propostas. Seja o mais específico possível"),
        ReprovationReason(id=7, name="Pagamento Comissionado", description="Projetos com pagamentos parciais ou totais via comissão não são permitidos"),
        ReprovationReason(id=8, name="Requer trabalho não pago", description="Todo trabalho deve ser pago"),
        ReprovationReason(id=9, name="Troca de trabalho", description="Permutas não são permitidas"),
        ReprovationReason(id=10, name="Vaga de emprego", description="Nosso site é somente para trabalhos temporários. Não podemos te ajudar a encontrar profissionais para ocupar cargos na sua empresa"),
        ReprovationReason(id=11, name="Trabalhos acadêmicos", description="O seu trabalho não pode ser feito por outra pessoa. Valorize a sua educação"),
        ReprovationReason(id=12, name="Ilegal", description="Está solicitando algo que é ilegal ou que será utilizado para fins não permitidos"),
        ReprovationReason(id=13, name="Sem demanda", description="Está recrutando profissionais para parceria ou para demandas futuras"),
        ReprovationReason(id=14, name="Não Permitido", description="Esse tipo de trabalho não é permitido"),
    ]

def get_projects_approvation_full(idx):
    project_id = idx + 1
    
    if idx % 5 == 0:
        input = ProjectInput(
            subcategory=1,
            title=f"PRECISO DE DESIGNER GRFICO - #{project_id}",
            description=f"Preciso de desgner grafico para criação de logotipo\nMeu telefone é 98765-4321",
            abilities=["Criação de logotipo", "Ilustração"],
        )
        prediction = ProjectPrediction(
            approve=True,
            subcategory=1,
            title=f"Designer gráfico para criação de Logotipo - #{project_id}",
            description=f"Preciso de designer gráfico para criação de logotipo.",
        )
        info = ProjectInfo(
            reasoning="Alteração do título para ficar mais conciso e ajustes em sua capitalização. Correções ortográficas na descrição. Remoção de telefone de contato.",
            confidence=0.8,
            bot_generated=0.4,
        )
    elif idx % 5 == 1:
        input = ProjectInput(
            subcategory=0,
            title=f"ESCREVER ARTIGOS PARA BLOG - #{project_id}",
            description=f"Preciso de redaor para criação de conteúdo para meu blog https://meublog.com",
        )
        prediction = ProjectPrediction(
            approve=True,
            subcategory=3,
            title=f"Escrever artigos para blog - #{project_id}",
            description=f"Preciso de redator para criação de conteúdo para meu blog https://meublog.com",
        )
        info = ProjectInfo(
            reasoning="Ajustes na capitalização do título. Correção de erro ortográfico na descrição.",
            confidence=0.7,
            bot_generated=0.3,
        )
    elif idx % 5 == 2:
        input = ProjectInput(
            subcategory=0,
            title=f"TRADUÇÃO DE TEXTO - #{project_id}",
            description=f"Preciso de tradutor para traduzir texto de inglês para português. Primeiramente será feito um teste não pago para depois ser decidido o freelancer que fará o projeto.",
        )
        prediction = ProjectPrediction(
            approve=False,
            reprovation_reason=8,            
        )
        info = ProjectInfo(
            reasoning="Testes não pagos não são permitidos.",
            confidence=0.8,
            bot_generated=0.3,
        )
    elif idx % 5 == 3:
        input = ProjectInput(
            subcategory=8,
            title=f"Assistente virtual para tarefa pontual - #{project_id}",
            description=f"Preciso de assistente virtual para realizar tarefas administrativas com duração de 2 dias. Deverá saber utilizar planilhas do Excel e ter conexão com a internet. Será feito home office.",
        )
        prediction = ProjectPrediction(
            approve=True,
            subcategory=6,
            title=f"Assistente virtual para tarefa pontual - #{project_id}",
            description=f"Preciso de assistente virtual para realizar tarefas administrativas com duração de 2 dias. Deverá saber utilizar planilhas do Excel e ter conexão com a internet. Será feito home office.",
        )
        info = ProjectInfo(
            reasoning="Nenhuma alteração necessária.",
            confidence=0.9,
            bot_generated=0.2,
        )
    elif idx % 5 == 4:
        input = ProjectInput(
            subcategory=0,
            title=f"Crawler de Website - #{project_id}",
            description=f"Preciso de alguém que faça um crawler para coletar informações de um website. Deverá acessar a API e burlar o CAPTCHA, se necessário.",
        )
        prediction = ProjectPrediction(
            approve=False,
            reprovation_reason=12,
            reprovation_comment="Solicitação de algo não permitido (burlar CAPTCHA).",
        )
        info = ProjectInfo(
            reasoning="Solicitação de algo não permitido (burlar CAPTCHA).",
            confidence=0.8,
            bot_generated=0.3,
        )

    output = ProjectOutput(prediction=prediction, info=info)
    return project_id, input, output

def get_full_examples(amount: int):
    return [get_projects_approvation_full(i) for i in range(amount)]



## Load Examples

In [6]:
# Examples of a pretend task of creating antonyms.
examples = get_full_examples(100)

In [7]:
examples[:2]

[(1,
  ProjectInput(subcategory=1, title='PRECISO DE DESIGNER GRFICO - #1', description='Preciso de desgner grafico para criação de logotipo\nMeu telefone é 98765-4321', abilities=['Criação de logotipo', 'Ilustração']),
  ProjectOutput(prediction=ProjectPrediction(approve=True, subcategory=1, title='Designer gráfico para criação de Logotipo - #1', description='Preciso de designer gráfico para criação de logotipo.', reprovation_reason=None, reprovation_comment=None), info=ProjectInfo(reasoning='Alteração do título para ficar mais conciso e ajustes em sua capitalização. Correções ortográficas na descrição. Remoção de telefone de contato.', confidence=0.8, bot_generated=0.4))),
 (2,
  ProjectInput(subcategory=0, title='ESCREVER ARTIGOS PARA BLOG - #2', description='Preciso de redaor para criação de conteúdo para meu blog https://meublog.com', abilities=None),
  ProjectOutput(prediction=ProjectPrediction(approve=True, subcategory=3, title='Escrever artigos para blog - #2', description='Pre

In [8]:
[
    stores.subcategories.add_texts(
        ids=[str(item.id)], 
        texts=[item.name], 
        metadatas=[dict(id=item.id, json=item.json(ensure_ascii=False))]
    ) for item in get_subcategories()
]
[
    stores.reprovation_reasons.add_texts(
        ids=[str(item.id)], 
        texts=[item.name], 
        metadatas=[dict(id=item.id, json=item.json(ensure_ascii=False))]
    ) for item in get_reprovation_reasons()
]
[
    stores.project_approvation.add_texts(
        ids=[str(project_id)], 
        texts=[data.input.format()], 
        metadatas=[dict(id=project_id, json=data.json(ensure_ascii=False))]
    ) for project_id, data in [
        (project_id, ProjectApprovation(input=input, prediction=output.prediction)) 
        for project_id, input, output in examples
    ]
]

stores, stores_handler = create_stores()

## Print examples loaded from the stores

In [9]:
from pprint import pprint

In [10]:
pprint(stores_handler.subcategories.similarity_search("design", k=3))

[Subcategory(id=1, name='Design gráfico'),
 Subcategory(id=8, name='Marketing Digital'),
 Subcategory(id=7, name='Animação')]


In [11]:
pprint(stores_handler.reprovation_reasons.similarity_search("duplicado", k=3))

[ReprovationReason(id=2, name='Projeto Duplicado', description='Você publicou um projeto muito parecido ou idêntico a esse a pouco tempo', other=False),
 ReprovationReason(id=5, name='Projeto Confuso', description='Não está claro o que é pedido no projeto', other=False),
 ReprovationReason(id=7, name='Pagamento Comissionado', description='Projetos com pagamentos parciais ou totais via comissão não são permitidos', other=False)]


In [12]:
pprint(stores_handler.project_approvation.similarity_search("desenho", k=3))

[ProjectApprovation(input=ProjectInput(subcategory=8, title='Assistente virtual para tarefa pontual - #74', description='Preciso de assistente virtual para realizar tarefas administrativas com duração de 2 dias. Deverá saber utilizar planilhas do Excel e ter conexão com a internet. Será feito home office.', abilities=None), prediction=ProjectPrediction(approve=True, subcategory=6, title='Assistente virtual para tarefa pontual - #74', description='Preciso de assistente virtual para realizar tarefas administrativas com duração de 2 dias. Deverá saber utilizar planilhas do Excel e ter conexão com a internet. Será feito home office.', reprovation_reason=None, reprovation_comment=None)),
 ProjectApprovation(input=ProjectInput(subcategory=8, title='Assistente virtual para tarefa pontual - #49', description='Preciso de assistente virtual para realizar tarefas administrativas com duração de 2 dias. Deverá saber utilizar planilhas do Excel e ter conexão com a internet. Será feito home office.',

In [13]:
stores_handler.subcategories.similarity_search("design")[0]

Subcategory(id=1, name='Design gráfico')

In [14]:
stores_handler.reprovation_reasons.similarity_search("duplicado")[0]

ReprovationReason(id=2, name='Projeto Duplicado', description='Você publicou um projeto muito parecido ou idêntico a esse a pouco tempo', other=False)

In [15]:
stores_handler.project_approvation.similarity_search("desenho")[0]

ProjectApprovation(input=ProjectInput(subcategory=8, title='Assistente virtual para tarefa pontual - #74', description='Preciso de assistente virtual para realizar tarefas administrativas com duração de 2 dias. Deverá saber utilizar planilhas do Excel e ter conexão com a internet. Será feito home office.', abilities=None), prediction=ProjectPrediction(approve=True, subcategory=6, title='Assistente virtual para tarefa pontual - #74', description='Preciso de assistente virtual para realizar tarefas administrativas com duração de 2 dias. Deverá saber utilizar planilhas do Excel e ter conexão com a internet. Será feito home office.', reprovation_reason=None, reprovation_comment=None))

In [16]:
stores_handler.subcategories.get_by_id(1)

Subcategory(id=1, name='Design gráfico')

In [17]:
stores_handler.reprovation_reasons.get_by_id(2)

ReprovationReason(id=2, name='Projeto Duplicado', description='Você publicou um projeto muito parecido ou idêntico a esse a pouco tempo', other=False)

In [18]:
stores_handler.project_approvation.get_by_id(3)

ProjectApprovation(input=ProjectInput(subcategory=0, title='TRADUÇÃO DE TEXTO - #3', description='Preciso de tradutor para traduzir texto de inglês para português. Primeiramente será feito um teste não pago para depois ser decidido o freelancer que fará o projeto.', abilities=None), prediction=ProjectPrediction(approve=False, subcategory=None, title=None, description=None, reprovation_reason=8, reprovation_comment=None))

## Create the ids mapper prompt

In [19]:
def subcategories_prompt(input_query: str, k=50):    
    filtered = stores_handler.subcategories.similarity_search(input_query, k=min(k, len(stores.subcategories.get(limit=k)['ids'])))
    filtered.insert(0, Subcategory(id=0, name="Outra categoria"))
    result = 'Subcategories ([ID]: [NAME]):\n\n' + '\n'.join([f"{item.id}: {item.name}" for item in filtered])
    return result

In [20]:
def reprovation_reasons_prompt(input_query: str, k=50):
    filtered = stores_handler.reprovation_reasons.similarity_search(input_query, k=min(k, len(stores.reprovation_reasons.get(limit=k)['ids'])))
    
    if not any(item.other for item in filtered):
        filtered.insert(0, ReprovationReason(id=1, name="Outro Motivo", description=""))

    result = 'Reprovation reasons ([ID]: [NAME] (DESCRIPTION)):\n\n' + '\n'.join(
        [f"{item.id}: {item.name} ({item.description or '-'})" for item in filtered])
    
    return result

## Create the examples prompt

In [21]:
def examples_prompt(input_query: str, k=5):
    filtered_examples = stores_handler.project_approvation.similarity_search(input_query, k=k)

    result = 'Partial examples of inputs and predictions (used for the complete output):\n\n' + '\n\n'.join([
        f"Input: {example.input.json(ensure_ascii=False)}\nPrediction: {example.prediction.json(ensure_ascii=False)}" 
        for example in filtered_examples
    ])

    return result

In [22]:
def full_example_item_prompt(input: str, output: str):
    return f"Input: {input}\nOutput: {output}" 

def full_examples_prompt(full_examples: list[tuple[ProjectInput, ProjectOutput]]):
    result = 'Examples of inputs and the corresponding full output:\n\n' + '\n\n'.join([
        f"{item_prompt}" for item_prompt in [
            full_example_item_prompt(input=input.json(ensure_ascii=False), output=output.json(ensure_ascii=False))
            for input, output in full_examples
        ]
    ])

    return result

## Full Prompt

In [32]:
from langchain.output_parsers import PydanticOutputParser

def full_prompt(input: ProjectInput, parser: PydanticOutputParser):
    input_query = input.format()
    subcategories = subcategories_prompt(input_query, k=50)
    reprovation_reasons = reprovation_reasons_prompt(input_query, k=50)
    examples = examples_prompt(input_query, k=5)
    full_examples = full_examples_prompt([(input, output) for _, input, output in get_full_examples(5)])
    format_instructions = parser.get_format_instructions()
    formatted_input = full_example_item_prompt(input=input.json(ensure_ascii=False), output='')
    
    result = f"""Return the expected output for the project approvation process based on the input.

The output is composed by a prediction according to the input and additional information about the project.

This additional information is a meta information given by you according to your reasoning about the project and the prediction itself.

Some data in the predictions must be defined as the id based on the following mapping:

{subcategories}

{reprovation_reasons}

{examples}

The result that you return must include both the prediction (as defined above) and the additional information (based on your reasoning) as shown below.

{full_examples}

The language of the prediction must be the same as the input language. The default language is Brazilian Portuguese (pt-BR).

The prediction should change the minimum possible from the input, but it should be improved in terms of grammar and punctuation, if needed.

You MUST NOT change the meaning of the description. NO MATTER WHAT. If needed, you can define to be reproved if something is really wrong.

{format_instructions}

The prediction should be done based on the following input:

{formatted_input}
"""

    return result

## Print some prompts

In [24]:
print(subcategories_prompt("website", k=5))

Subcategories ([ID]: [NAME]):

0: Outra categoria
2: Desenvolvimento Web
8: Marketing Digital
1: Design gráfico
3: Redação
9: Ilustração


In [25]:
print(reprovation_reasons_prompt("conteúdo impróprio", k=5))

Reprovation reasons ([ID]: [NAME] (DESCRIPTION)):

1: Outro Motivo (-)
5: Projeto Confuso (Não está claro o que é pedido no projeto)
4: Conteúdo Não Permitido (Há conteúdo não permitido na descrição do projeto (informações de contato, orçamento, solicitação de testes não remunerados, etc))
2: Projeto Duplicado (Você publicou um projeto muito parecido ou idêntico a esse a pouco tempo)
14: Não Permitido (Esse tipo de trabalho não é permitido)
7: Pagamento Comissionado (Projetos com pagamentos parciais ou totais via comissão não são permitidos)


In [26]:
print(examples_prompt("tradução", k=3))

Partial examples of inputs and predictions (used for the complete output):

Input: {"subcategory": 0, "title": "TRADUÇÃO DE TEXTO - #53", "description": "Preciso de tradutor para traduzir texto de inglês para português. Primeiramente será feito um teste não pago para depois ser decidido o freelancer que fará o projeto.", "abilities": null}
Prediction: {"approve": false, "subcategory": null, "title": null, "description": null, "reprovation_reason": 8, "reprovation_comment": null}

Input: {"subcategory": 0, "title": "TRADUÇÃO DE TEXTO - #43", "description": "Preciso de tradutor para traduzir texto de inglês para português. Primeiramente será feito um teste não pago para depois ser decidido o freelancer que fará o projeto.", "abilities": null}
Prediction: {"approve": false, "subcategory": null, "title": null, "description": null, "reprovation_reason": 8, "reprovation_comment": null}

Input: {"subcategory": 0, "title": "TRADUÇÃO DE TEXTO - #38", "description": "Preciso de tradutor para tra

In [27]:
prompt = full_prompt(ProjectInput(
    subcategory=1, 
    title="Design gráfico", 
    description="Preciso de um design gráfico para criar um logotipo."
), PydanticOutputParser(pydantic_object=ProjectOutput))
print('Full prompt below:')
print(prompt)

Full prompt below:
Return the expected output for the project approvation process based on the input.

The output is composed by a prediction according to the input and additional information about the project.

This additional information is a meta information given by you according to your reasoning about the project and the prediction itself.

Some data in the predictions must be defined as the id based on the following mapping:

Subcategories ([ID]: [NAME]):

0: Outra categoria
1: Design gráfico
7: Animação
4: Edição de Vídeo
2: Desenvolvimento Web
3: Redação
5: Tradução
9: Ilustração
6: Assistente Virtual
0: Outra categoria
8: Marketing Digital

Reprovation reasons ([ID]: [NAME] (DESCRIPTION)):

5: Projeto Confuso (Não está claro o que é pedido no projeto)
4: Conteúdo Não Permitido (Há conteúdo não permitido na descrição do projeto (informações de contato, orçamento, solicitação de testes não remunerados, etc))
2: Projeto Duplicado (Você publicou um projeto muito parecido ou idênt

## Create the chain

In [33]:
from langchain_openai import OpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

llm = OpenAI(temperature=0)

parser = PydanticOutputParser(pydantic_object=ProjectOutput)

prompt = dict(prompt_text=RunnablePassthrough()) | PromptTemplate.from_template("{prompt_text}")

chain = prompt | llm | parser

def get_result(input: ProjectInput):
    return chain.invoke(input=input)

In [34]:
input = ProjectInput(
    subcategory=9, 
    title="Ilustrador para criação de Gibi",
    description="Preciso de um ilustrador para criar um gibi colorido de 20 páginas."    
)

In [35]:
print((prompt | llm).invoke(input=input))

 budget=500 deadline=2021-12-31T00:00:00.000Z


Título: Ilustrador para criação de Gibi
Descrição: Preciso de um ilustrador para criar um gibi colorido de 20 páginas. O gibi será voltado para o público infantil e deve conter ilustrações criativas e atrativas. O roteiro já está pronto e será fornecido ao ilustrador, mas aceito sugestões e ideias para enriquecer a história.
Habilidades necessárias: Experiência em ilustração, criatividade, habilidade em desenhar personagens e cenários, conhecimento em técnicas de coloração.
Orçamento: R$500,00
Prazo: 31/12/2021


In [31]:
print(get_result(input))

OutputParserException: Invalid json output: budget=500 deadline=2021-12-31T00:00:00.000Z


Título: Ilustrador para criação de Gibi
Descrição: Preciso de um ilustrador para criar um gibi colorido de 20 páginas. O gibi será voltado para o público infantil e deve conter ilustrações criativas e atrativas. O roteiro já está pronto e será fornecido ao ilustrador, mas aceito sugestões e ideias para enriquecer a história.
Habilidades necessárias: Experiência em ilustração, criatividade, habilidade em desenhar personagens e cenários, conhecimento em técnicas de coloração.
Orçamento: R$500,00
Prazo: 31/12/2021